In [1]:
## Ran for eveything now with excluding the gauges that we don't want

# Create a set of dimensionless profiles
Read in all of the events, for all durations, for all gauges, for all ensemble members.  
Convert them to dimensionless profiles, with 12 values between 0 and 1.  
Each value is a dimensionless, cumulative rainfall value (cumulative rainfall at this timestep, normalised by the total event rainfall):
- 0 means no rainfall has occurred, and 
- 1 means the total event rainfall has been reached.  

If there are less than 12 values, then these are filled in with interpolation

In [6]:
def find_quintile_with_max_cumulative_rainfall(cumulative_rainfall):
    total_rainfall = cumulative_rainfall[-1]  # Total cumulative rainfall at the end
    total_time = len(cumulative_rainfall)  # Total time steps

    # Calculate the time index for each quintile
    quintile_times = np.linspace(0, total_time, 6, dtype=int)  # Divide into 5 equal parts

    # Calculate cumulative rainfall in each quintile
    quintile_rainfall = np.zeros(5)
    for i in range(5):
        start_idx = quintile_times[i]
        end_idx = quintile_times[i + 1] if i < 4 else total_time
        quintile_rainfall[i] = cumulative_rainfall[end_idx - 1] - cumulative_rainfall[start_idx]

    # Find the quintile with the maximum cumulative rainfall
    max_quintile = np.argmax(quintile_rainfall)

    # Return the quintile index (1-indexed)
    return max_quintile + 1

def process_file(gauge_num, fp):
    # Read the CSV file
    test = pd.read_csv(fp)
    
    # Find duration from file name
    pattern = re.compile(r'(\d+\.?\d*)hrs')
    match = pattern.search(file)
    if match:
        duration = match.group(1)
    else:
        duration = None
    
    precipitation_sum = test['precipitation (mm)'].sum()

    if len(test) == 1:
        print(f"Only 1 value at gauge_num {gauge_num} for {file}")
        return None, None, None
    else:
        times = np.array(range(0, len(test)))
        normalized_time, normalized_rainfall = create_dimensionless_profile(times, np.array(test['precipitation (mm/hr)']))
        max_quintile = find_quintile_with_max_cumulative_rainfall(normalized_rainfall)
        # print(max_quintile)
        interpolated_rainfall = interpolate_and_bin(normalized_time, normalized_rainfall)
        # heaviest_segment = categorize_normalized_rainstorm(interpolated_rainfall)
        real_duration = len(test)/2
    return interpolated_rainfall, normalized_rainfall, duration, real_duration, precipitation_sum, max_quintile    

def check_for_nan(profiles_list, dimensionless_profiles_list, durations_for_nimrod_profiles,real_durations_for_nimrod_profiles,volumes_for_nimrod_profiles, max_quintiles_ls):
    new_profiles_ls = []
    new_dimensionless_profiles_ls = []
    new_real_durations_ls = []
    new_durations_ls = []
    new_volumes_ls = []
    new_max_quintile_ls = []
    
    for i, profile in enumerate(profiles_list):
        if np.isnan(profile).any():
            print(f"NaN values found in profile {i}")
        else:
            new_profiles_ls.append(profile)
            new_durations_ls.append(durations_for_nimrod_profiles[i])
            new_real_durations_ls.append(volumes_for_nimrod_profiles[i])
            new_volumes_ls.append(volumes_for_nimrod_profiles[i])
            new_dimensionless_profiles_ls.append(dimensionless_profiles_list[i])
            new_max_quintile_ls.append(max_quintiles_ls[i])
            
    return new_profiles_ls, new_dimensionless_profiles_ls, new_durations_ls,new_real_durations_ls, new_volumes_ls,new_max_quintile_ls

In [3]:
gauge_nums = range(0,1263)

# NIMROD
### Make profiles

In [9]:
for nimrod_option in ["NIMROD_2.2km_filtered_100","NIMROD_2.2km_filtered_300"]:
    print(nimrod_option)

    # Initialize lists to store results
    nimrod_profiles = []
    durations_for_nimrod_profiles = []
    real_durations_for_nimrod_profiles = []
    volumes_for_nimrod_profiles = []
    dimensionless_profiles = [] 
    max_quintiles = []

    # Process each gauge
    for gauge_num in range(0,1269):
        # the gauges that didn't work for finding events
        if gauge_num not in [423, 444, 827, 888]:
            # Excluding ones near windfarms
            if gauge_num not in [27,36,57,61,75,97,101,106,120,132,190,204, 239, 285,348,376]:
                if gauge_num % 100 == 0:
                    print(gauge_num)

                # Create a list of all the event CSVs
                files = [f for f in os.listdir(f"../../../ProcessedData/IndependentEvents/NIMROD/{nimrod_option}/{gauge_num}/") if f.endswith('.csv')]
                files = np.sort(files)

                # Process each file
                for file in files:
                    fp = f"../../../ProcessedData/IndependentEvents/NIMROD/{nimrod_option}/{gauge_num}/{file}"
                    interpolated_rainfall, normalized_rainfall, duration, real_duration, precipitation_sum, max_quintile  =  process_file(gauge_num, fp)
                    
                    if interpolated_rainfall is not None and duration is not None and precipitation_sum is not None:
                        nimrod_profiles.append(interpolated_rainfall)
                        durations_for_nimrod_profiles.append(duration)
                        real_durations_for_nimrod_profiles.append(real_duration)
                        volumes_for_nimrod_profiles.append(precipitation_sum)
                        dimensionless_profiles.append(normalized_rainfall)
                        max_quintiles.append(max_quintile)

    ## Remove profiles containing NANs
    nimrod_profiles, dimensionless_profiles, durations_for_nimrod_profiles, real_durations_for_nimrod_profiles, volumes_for_nimrod_profiles, max_quintiles =  check_for_nan(nimrod_profiles, dimensionless_profiles, durations_for_nimrod_profiles, real_durations_for_nimrod_profiles, volumes_for_nimrod_profiles, max_quintiles)
    names =['profiles', "dimensionless_profiles",'durations_for_profiles',  'real_durations_for_profiles','volumes_for_profiles','max_quintiles']
    
    for number, file in enumerate([nimrod_profiles, dimensionless_profiles, durations_for_nimrod_profiles, real_durations_for_nimrod_profiles,volumes_for_nimrod_profiles, max_quintiles]):
        file_name = names[number]
        cache_filepath = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/NIMROD/{nimrod_option}_{file_name}.pkl"
        with open(cache_filepath, 'wb') as f:
            pickle.dump(file, f)

NIMROD_2.2km_filtered_100
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
NaN values found in profile 19653
NaN values found in profile 20092
NaN values found in profile 26755
NaN values found in profile 26770
NaN values found in profile 26785
NaN values found in profile 26800
NaN values found in profile 26817
NaN values found in profile 26832
NaN values found in profile 26847
NaN values found in profile 27832
NaN values found in profile 27847
NaN values found in profile 27863
NaN values found in profile 27879
NaN values found in profile 27897
NaN values found in profile 27912
NaN values found in profile 27927
NaN values found in profile 29016
NaN values found in profile 29031
NaN values found in profile 29046
NaN values found in profile 29062
NaN values found in profile 29078
NaN values found in profile 29093
NaN values found in profile 29108
NaN values found in profile 31073
NaN values found in profile 31088
NaN values found in profile 31103
NaN values found in profile 31139
NaN

# UKCP18
### Make and pickle profiles

In [50]:
# Initialize lists to store results
model_profiles = []
durations_for_model_profiles = []
real_durations_for_model_profiles = []
volumes_for_model_profiles = []
normalised_rainfalls = []
heaviest_segments = []

# Process each gauge
em='bc005'
for gauge_num in gauge_nums:
    # the gauges that didn't work for finding events
    if gauge_num not in [444, 827, 888]:
        # Excluding ones near windfarms
        if gauge_num not in [27,36,57,61,75,97,101,106,120,132,190,204, 239, 285,348,376]:
            if gauge_num % 100 == 0:
                print(gauge_num)
            # Create a list of all the event CSVs
            files = [f for f in os.listdir(f"../../ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/") if f.endswith('.csv')]
            files = np.sort(files)

            # Process each file
            for file in files:
                fp = f"../../ProcessedData/IndependentEvents/UKCP18_30mins/{em}/{gauge_num}/{file}"
                interpolated_rainfall, duration, real_duration, precipitation_sum, normalised_rainfall = process_file(gauge_num, fp)
                if interpolated_rainfall is not None and duration is not None and precipitation_sum is not None:
                    model_profiles.append(interpolated_rainfall)
                    durations_for_model_profiles.append(duration)
                    real_durations_for_model_profiles.append(real_duration)
                    volumes_for_model_profiles.append(precipitation_sum)
                    normalised_rainfalls.append(normalised_rainfall)
                    heaviest_segments.append(categorize_normalized_rainstorm(interpolated_rainfall))
                
## Remove profiles containing NANs
names =['model_profiles', 'model_durations_for_profiles','model_real_durations_for_profiles','model_volumes_for_profiles', 'heaviest_segments']
for number, file in enumerate([model_profiles, durations_for_model_profiles, real_durations_for_model_profiles, volumes_for_model_profiles, heaviest_segments]):
    file_name = names[number]
    cache_filepath = f"/nfs/a319/gy17m2a/PhD/ProcessedData/Profiles/UKCP18_30mins/{em}/{file_name}.pkl"
    with open(cache_filepath, 'wb') as f:
        pickle.dump(file, f)                


0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [51]:
print(len(model_profiles))
print(len(durations_for_model_profiles))
print(len(volumes_for_model_profiles))
print(len(real_durations_for_model_profiles))

121063
121063
121063
121063
